## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# ImportDependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
ls

Vacation_Search.ipynb       __pycache__/
WeatherPy_vacation.csv      config.py
WeatherPy_vacation_map.png


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,83,40,11.50,scattered clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-1.44,76,0,5.70,clear sky
2,2,Hokitika,NZ,-42.7167,170.9667,54.79,73,99,15.90,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,80.28,76,45,19.08,scattered clouds
4,4,East London,ZA,-33.0153,27.9116,71.60,78,20,11.50,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 55
What is your desired maximum temperature for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,83,40,11.50,scattered clouds
3,3,Butaritari,KI,3.0707,172.7902,80.28,76,45,19.08,scattered clouds
4,4,East London,ZA,-33.0153,27.9116,71.60,78,20,11.50,few clouds
5,5,Hobart,AU,-42.8794,147.3294,61.00,59,90,12.66,overcast clouds
11,11,Carnarvon,AU,-24.8667,113.6333,82.40,89,35,6.91,scattered clouds
12,12,Hilo,US,19.7297,-155.0900,69.80,83,90,5.75,overcast clouds
13,13,Bubaque,GW,11.2833,-15.8333,77.86,55,84,7.05,broken clouds
14,14,Mar Del Plata,AR,-38.0023,-57.5575,66.00,72,75,14.97,broken clouds
15,15,Port Elizabeth,ZA,-33.9180,25.5701,69.80,73,75,8.05,broken clouds
19,19,Begang,PH,6.6909,122.0168,75.96,91,100,6.22,moderate rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,81.00,83,40,11.50,scattered clouds
3,3,Butaritari,KI,3.0707,172.7902,80.28,76,45,19.08,scattered clouds
4,4,East London,ZA,-33.0153,27.9116,71.60,78,20,11.50,few clouds
5,5,Hobart,AU,-42.8794,147.3294,61.00,59,90,12.66,overcast clouds
11,11,Carnarvon,AU,-24.8667,113.6333,82.40,89,35,6.91,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
665,665,Atar,MR,20.5169,-13.0499,75.11,15,100,14.74,overcast clouds
668,668,Bontang,ID,0.1333,117.5000,76.14,87,99,4.50,overcast clouds
671,671,Ixtapa,MX,20.7000,-105.2000,71.60,56,20,3.91,few clouds
672,672,Mangan,IN,27.5167,88.5333,56.08,38,3,3.38,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,81.00,scattered clouds,-20.3428,57.7575,
3,Butaritari,KI,80.28,scattered clouds,3.0707,172.7902,
4,East London,ZA,71.60,few clouds,-33.0153,27.9116,
5,Hobart,AU,61.00,overcast clouds,-42.8794,147.3294,
11,Carnarvon,AU,82.40,scattered clouds,-24.8667,113.6333,
12,Hilo,US,69.80,overcast clouds,19.7297,-155.0900,
13,Bubaque,GW,77.86,broken clouds,11.2833,-15.8333,
14,Mar Del Plata,AR,66.00,broken clouds,-38.0023,-57.5575,
15,Port Elizabeth,ZA,69.80,broken clouds,-33.9180,25.5701,
19,Begang,PH,75.96,moderate rain,6.6909,122.0168,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,81.00,scattered clouds,-20.3428,57.7575,Casa Tia Villa
3,Butaritari,KI,80.28,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
4,East London,ZA,71.60,few clouds,-33.0153,27.9116,Tu Casa
5,Hobart,AU,61.00,overcast clouds,-42.8794,147.3294,Mantra on Collins Hobart
11,Carnarvon,AU,82.40,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [13]:
# 11a. Add a marker layer for each city to the map. 
import gmaps.datasets
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))